# Connect to RDS

In [29]:
!pip3 install pymysql==0.9.3

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [30]:
import pymysql
import boto3
import pandas as pd
import numpy as np
import s3fs

In [31]:
connection = pymysql.connect( 'database-1.cbrnhoat32p7.us-east-1.rds.amazonaws.com', 'myuser', 'myPass05')
cursor = connection.cursor()
print(cursor)

In [32]:
cursor.execute("select version()")

1

In [33]:
data = cursor.fetchone()

In [34]:
data

('8.0.28',)

In [35]:
sql = '''select database collections'''


In [36]:
cursor.connection.commit()

In [38]:
sql = '''use collections''' 
cursor.execute(sql)

0

In [39]:
#show the tables in RDs
sql = '''
show tables
'''
cursor.execute(sql)


1

In [40]:
cursor.fetchall()

(('items',),)

In [71]:
sql = '''select * from collections.items'''
cursor.execute(sql)
df = pd.read_sql_query(sql, connection)
df

/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,CouponsDesc,CouponsSummery,CouponPrice
0,Cats vs Pickles - Limited Edition Holiday 12pk...,https://m.media-amazon.com/images/I/51n8ENJFUW...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,282.0,54.99,59.99,None,None,54.990
1,Osmo - Super Studio Disney Frozen 2 - Ages 5-1...,https://m.media-amazon.com/images/I/91SJQup1fq...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,4333.0,29.99,29.99,None,None,29.990
2,Pop it Pink Ombre Cupcake Fidget Toy Girls and...,https://m.media-amazon.com/images/I/81G2b5S-wQ...,5.0 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,1.0,7.99,7.99,None,None,7.990
3,"Jasonwell Splash Pad Sprinkler for Kids 68"" Sp...",https://m.media-amazon.com/images/I/81s0fdVXUu...,4.4 out of 5 stars,https://www.amazon.com/Jasonwell-Sprinkle-Spri...,11005.0,19.99,38.99,10% coupon applied at checkoutSave 10% with c...,Save 10%,17.991
4,Magnetic Tiles Toys for 3 4 5 6 7 8+ Year Old ...,https://m.media-amazon.com/images/I/71DQtErS2T...,4.6 out of 5 stars,https://www.amazon.com/Magnetic-Building-Creat...,1991.0,25.97,35.99,20% coupon applied at checkoutSave 20% with c...,Save 20%,20.776
...,...,...,...,...,...,...,...,...,...,...
142,ECTY Unicorn Gifts Lamps Night Lights for Kids...,https://m.media-amazon.com/images/I/81gn4sWprG...,4.4 out of 5 stars,https://www.amazon.com/LONGXU-Unicorn-Changing...,2089.0,15.99,26.99,Lowest price in 30 days,None,15.990
143,"The Original Slinky Walking Spring Toy, Metal ...",https://m.media-amazon.com/images/I/91jLIfnfDn...,4.5 out of 5 stars,https://www.amazon.com/Slinky-60100-Original-B...,23933.0,3.50,3.50,50% off gift wrap service: code GIFTWRAP50,None,3.500
144,LEGO Speed Champions Aston Martin Valkyrie AMR...,https://m.media-amazon.com/images/I/818L58F+ei...,4.8 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,65.0,39.97,39.97,None,None,39.970
145,Dinosaur Toy Truck for Kids 3-7 with Flash Lig...,https://m.media-amazon.com/images/I/81aDx6A2XT...,4.5 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,78.0,24.99,24.99,None,None,24.990


In [77]:
df['PercentProfit'] = df.apply(lambda row: abs(((row.CouponPrice - row.OrgPrice)/row.OrgPrice) * 100), axis=1)
#df['PercentProfit'] = abs(((df["CouponPrice"] - df["OrgPrice"])/df["OrgPrice"]) * 100)
df

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,CouponsDesc,CouponsSummery,CouponPrice,PercentProfit
0,Cats vs Pickles - Limited Edition Holiday 12pk...,https://m.media-amazon.com/images/I/51n8ENJFUW...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,282.0,54.99,59.99,None,None,54.990,8.334722
1,Osmo - Super Studio Disney Frozen 2 - Ages 5-1...,https://m.media-amazon.com/images/I/91SJQup1fq...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,4333.0,29.99,29.99,None,None,29.990,0.000000
2,Pop it Pink Ombre Cupcake Fidget Toy Girls and...,https://m.media-amazon.com/images/I/81G2b5S-wQ...,5.0 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,1.0,7.99,7.99,None,None,7.990,0.000000
3,"Jasonwell Splash Pad Sprinkler for Kids 68"" Sp...",https://m.media-amazon.com/images/I/81s0fdVXUu...,4.4 out of 5 stars,https://www.amazon.com/Jasonwell-Sprinkle-Spri...,11005.0,19.99,38.99,10% coupon applied at checkoutSave 10% with c...,Save 10%,17.991,53.857399
4,Magnetic Tiles Toys for 3 4 5 6 7 8+ Year Old ...,https://m.media-amazon.com/images/I/71DQtErS2T...,4.6 out of 5 stars,https://www.amazon.com/Magnetic-Building-Creat...,1991.0,25.97,35.99,20% coupon applied at checkoutSave 20% with c...,Save 20%,20.776,42.272854
...,...,...,...,...,...,...,...,...,...,...,...
142,ECTY Unicorn Gifts Lamps Night Lights for Kids...,https://m.media-amazon.com/images/I/81gn4sWprG...,4.4 out of 5 stars,https://www.amazon.com/LONGXU-Unicorn-Changing...,2089.0,15.99,26.99,Lowest price in 30 days,None,15.990,40.755835
143,"The Original Slinky Walking Spring Toy, Metal ...",https://m.media-amazon.com/images/I/91jLIfnfDn...,4.5 out of 5 stars,https://www.amazon.com/Slinky-60100-Original-B...,23933.0,3.50,3.50,50% off gift wrap service: code GIFTWRAP50,None,3.500,0.000000
144,LEGO Speed Champions Aston Martin Valkyrie AMR...,https://m.media-amazon.com/images/I/818L58F+ei...,4.8 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,65.0,39.97,39.97,None,None,39.970,0.000000
145,Dinosaur Toy Truck for Kids 3-7 with Flash Lig...,https://m.media-amazon.com/images/I/81aDx6A2XT...,4.5 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,78.0,24.99,24.99,None,None,24.990,0.000000


In [78]:
df.columns

Index(['Title', 'Image', 'Rating', 'ProdLink', 'TotalReviews', 'Price',
       'OrgPrice', 'CouponsDesc', 'CouponsSummery', 'CouponPrice',
       'PercentProfit'],
      dtype='object')

In [79]:
df=df.sort_values(by='PercentProfit', ascending=False)
df

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,CouponsDesc,CouponsSummery,CouponPrice,PercentProfit
51,TEMI 8 Channels 2.4G Remote Control Dinosaur T...,https://m.media-amazon.com/images/I/71CfgcPxmT...,4.4 out of 5 stars,https://www.amazon.com/TEMI-Electronic-Educati...,4809.0,39.99,89.90,$6.00 coupon applied at checkoutSave $6.00 wi...,Save $6.00,33.9900,62.191324
79,Baby Toys 6 to 12 Months Touch & Go Musical Li...,https://m.media-amazon.com/images/I/71QyNYo+-v...,4.5 out of 5 stars,https://www.amazon.com/Dinosaur-Electronic-Cra...,2653.0,18.69,44.99,None,None,18.6900,58.457435
3,"Jasonwell Splash Pad Sprinkler for Kids 68"" Sp...",https://m.media-amazon.com/images/I/81s0fdVXUu...,4.4 out of 5 stars,https://www.amazon.com/Jasonwell-Sprinkle-Spri...,11005.0,19.99,38.99,10% coupon applied at checkoutSave 10% with c...,Save 10%,17.9910,53.857399
10,"HOLA Baby Toys 6 to 12 Months, Baby Rattles Ac...",https://m.media-amazon.com/images/I/61wtIWS4Ql...,4.8 out of 5 stars,https://www.amazon.com/HOLA-Rattles-Activity-C...,1035.0,14.97,29.99,1 Count (Pack of 1),Save 5%,14.2215,52.579193
24,"Bravokids Toys for 3-6 Years Old Girls Boys, L...",https://m.media-amazon.com/images/I/61Sz5N89C4...,4.5 out of 5 stars,https://www.amazon.com/Bravokids-Writing-Elect...,19023.0,22.99,29.99,37% coupon applied at checkoutSave 37% with c...,Save 37%,14.4837,51.704902
...,...,...,...,...,...,...,...,...,...,...,...
39,Fisher-Price Rock-a-Stack and Baby's First Blo...,https://m.media-amazon.com/images/I/81HB7ARpuU...,4.8 out of 5 stars,https://www.amazon.com/Fisher-Price-Stack-Bloc...,19579.0,14.99,14.99,50% off gift wrap service: code GIFTWRAP50,None,14.9900,0.000000
116,"The Original Slinky Walking Spring Toy, Metal ...",https://m.media-amazon.com/images/I/91jLIfnfDn...,4.5 out of 5 stars,https://www.amazon.com/Slinky-60100-Original-B...,23933.0,3.50,3.50,50% off gift wrap service: code GIFTWRAP50,None,3.5000,0.000000
117,LEGO Speed Champions Aston Martin Valkyrie AMR...,https://m.media-amazon.com/images/I/818L58F+ei...,4.8 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,65.0,39.97,39.97,None,None,39.9700,0.000000
83,"TEUVO Pet Care Play Set Doctor Kit for Kids, 1...",https://m.media-amazon.com/images/I/71TBIlpuQ9...,4.3 out of 5 stars,https://www.amazon.com/TEUVO-Pretend-Grooming-...,772.0,29.89,29.89,Join Prime to save $4.18 on this item,None,29.8900,0.000000


In [69]:
#(100- (CouponPrice/OrgPrice*100)) percentage reduction
#df3=df.assign[newcol]=df['CouponPrice'] + df['OrgPrice']
df['pricereduction'] = df.apply(lambda row: row.OrgPrice + row.OrgPrice)

AttributeError: 'Series' object has no attribute 'OrgPrice'

In [42]:
sql='''


SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME='items' '''
cursor.execute(sql)

10

In [26]:
sql = '''
insert into items(Title, Image,Rating,ProdLink,TotalReviews, Price,OrgPrice,CouponDesc) values('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')''' % ('Added new toy', 'https://media.istockphoto.com/photos/creating-new-cities-picture-id539350870?s=612x612', '4.7 out of 5 stars', 'https://www.amazon.com/gp/slredirect/picassoRe...', '100', '$80.00', '$99.99','save $4')

cursor.execute(sql)
cursor.connection.commit()

InternalError: (1054, "Unknown column 'CouponDesc' in 'field list'")

In [44]:
sql = '''select * from items'''
cursor.execute(sql)
cursor.connection.commit()